IMPORT


In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import math


from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.api import OLS
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000


#path = '/Koding_With_Kolesh/challenges/RMB_NOWCAST_APRIL/'
cpi = pd.read_csv('CPI_Historic_Values_Zindi_May_23.csv')
#vehicles = pd.read_csv('Naamsa_Vehicle_Sales.csv')
petrol = pd.read_csv('petrolem.csv')
lending = pd.read_csv('historical_prime_lending_rates.csv')
lending = lending.ffill()
seed = 3

PIVOTING THE DATA


In [29]:
cpi_pivot = cpi.pivot(index = 'Month', columns = 'Category', values = 'Value').reset_index()#changing from a long format to a wide format,
cpi_pivot['Month'] = pd.to_datetime(cpi_pivot['Month'])
cpi_pivot = cpi_pivot.sort_values("Month").reset_index(drop=True)

ADDING JUNE and july'S DATA MANUALLY FROM https://www.statssa.gov.za/publications/P0141/P0141June2023.pdf and https://www.statssa.gov.za/publications/P0141/P0141July2023.pdf


In [30]:
date_str = '2023-06-30'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)
cpi_of_june = [110.9, 104.3, 99.6, 110.4, 118.3, 109.8, 110.8, 107.7, 105.4, 109.6, 105.3, 110.0, 112.3]

for i, col in enumerate(cpi_pivot.columns[1:]):
    cpi_pivot.at[17, col] = cpi_of_june[i]
date_str = '2023-07-31'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)
cpi_of_july = [111.5, 104.5, 99.5, 110.4, 118.5, 110.8, 110.6, 108.2 , 108.4, 109.9, 105.5, 110, 112.6]

for i, col in enumerate(cpi_pivot.columns[1:]):
    cpi_pivot.at[18, col] = cpi_of_july[i]

ADD THE August ROW


In [31]:
date_str = '2023-08-31'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)

FEATURE ENGINNERING


In [32]:
feats_to_lag = cpi_pivot.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,5):
        cpi_pivot[f'prev_{i}_month_{col}'] = cpi_pivot[col].shift(i)
columns = list(cpi_pivot.columns)
columns = [item for item in columns if "Transport" not in item]

In [33]:
petrol['Month'] = pd.to_datetime(petrol['Month'])
petrol = petrol.sort_values("Month").reset_index(drop=True)
feats_to_lag = petrol.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,5):
        petrol[f'prev_{i}_month_{col}'] = petrol[col].shift(i)
start_date = pd.to_datetime('2022-01-31')
end_date = pd.to_datetime('2023-08-31')
petrol = petrol[(petrol['Month'] >= start_date) & (petrol['Month'] <= end_date)]
cpi_pivot = pd.merge(cpi_pivot, petrol, on='Month', suffixes=('_df1', '_df2'))

In [34]:
lending['Interest_per_annum'] = lending['Interest_per_annum'].str.rstrip('%').astype(float).astype(int)
lending['Month'] = pd.to_datetime(lending['Month'])
lending = lending.sort_values("Month").reset_index(drop=True)
feats_to_lag = lending.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,5):
        lending[f'prev_{i}_month_{col}'] = lending[col].shift(i)
start_date = pd.to_datetime('2022-01-31')
end_date = pd.to_datetime('2023-08-31')
lending = lending[(lending['Month'] >= start_date) & (lending['Month'] <= end_date)]
cpi_pivot = pd.merge(cpi_pivot, lending, on='Month', suffixes=('_df1', '_df2'))

In [35]:
petcol = list(petrol.columns)
search_strings = ["petrol95" ,"randuS_exchenge" ]
petcol = [item for item in petcol if all(search_str not in item for search_str in search_strings)]
petcol = petcol[1:]

HANDLE MISSING DATA


In [36]:
cpi_pivot = cpi_pivot.drop(0)
cpi_pivot = cpi_pivot.bfill()

TRAIN AND VALIDATION


In [37]:
train = cpi_pivot[cpi_pivot['Month'] != "2023-08-31"]
test = cpi_pivot[cpi_pivot['Month'] == "2023-08-31"]

training_set = train[train['Month']!= '2023-07-31']
validation_set = train[train['Month']== '2023-07-31']

train.shape, test.shape, training_set.shape, validation_set.shape

((18, 101), (1, 101), (17, 101), (1, 101))

MODELING


In [38]:
target_cols = ['Alcoholic beverages and tobacco', 'Clothing and footwear',
       'Communication', 'Education', 'Food and non-alcoholic beverages',
       'Headline_CPI', 'Health', 'Household contents and services',
       'Housing and utilities', 'Miscellaneous goods and services',
       'Recreation and culture', 'Restaurants and hotels ', 'Transport']

#if you add additional data sources that have no value in the predicting month , drop it, now that you have their lags
features= [col for col in train.columns if col not in target_cols + ['Month']]
featurest= [col for col in train.columns if col not in columns + ['Transport']]

X_train = training_set[features]
X_traint = training_set[featurest]
y_train = training_set[target_cols]

X_val = validation_set[features]
X_valt = validation_set[featurest]
y_val = validation_set[target_cols]

l_models = {}

y_predl = []

rmsel_dict = {} 

scaler = MinMaxScaler()

#training
for target_col in target_cols:
    if target_col == "Transport":
        #l_model = Lasso(alpha=0.1, random_state=seed)
        l_model = XGBRegressor(seed=seed)
        X_train_scaled = scaler.fit_transform(X_traint)

        l_model.fit(X_train_scaled, y_train[target_col])

        l_models[target_col] = l_model
        
        l_model = l_models[target_col]

        X_val_scaled = scaler.transform(X_valt)

        y_pred_coll = l_model.predict(X_val_scaled)

        rmsel_col = np.sqrt(mean_squared_error(y_pred_coll, y_val[target_col]))
        rmsel_dict[target_col] = rmsel_col 

        y_predl.append(y_pred_coll)
    else:
        #l_model = Lasso(alpha=0.1, random_state=seed)
        l_model = XGBRegressor(seed=seed)
        X_train_scaled = scaler.fit_transform(X_train)

        l_model.fit(X_train_scaled, y_train[target_col])

        l_models[target_col] = l_model
        
        l_model = l_models[target_col]

        X_val_scaled = scaler.transform(X_val)

        y_pred_coll = l_model.predict(X_val_scaled)

        rmsel_col = np.sqrt(mean_squared_error(y_pred_coll, y_val[target_col]))
        rmsel_dict[target_col] = rmsel_col 

        y_predl.append(y_pred_coll)

# scoring
y_predl = np.array(y_predl).T
y_predl[:,5][0] = ((6.26 * y_predl[:,0][0]) + (3.65 * y_predl[:,1][0]) + (2.42 * y_predl[:,2][0]) 
                   + (2.62 * y_predl[:,3][0]) + (17.14 * y_predl[:,4][0]) + (1.44 * y_predl[:,6][0]) + 
                   (4.37 * y_predl[:,7][0]) 
                   + (24.49 * y_predl[:,8][0]) + (14.81 * y_predl[:,9][0]) + (5.2 * y_predl[:,10][0]) + 
                   (3.25 * y_predl[:,11][0]) + (14.35 * y_predl[:,12][0]))/100

dfl = pd.DataFrame({'y_pred': y_predl.flatten(), 'y_val': y_val.values.flatten()})

# Print RMSE for each target column
for target_col in target_cols:
    print(f'RMSE of Lasso Regression for {target_col}: {rmsel_dict[target_col]}')

# Calculate the average RMSE across all target columns
average_rmse = np.mean(list(rmsel_dict.values()))
print(f'Average RMSE of Lasso Regression: {average_rmse}')

RMSE of Lasso Regression for Alcoholic beverages and tobacco: 0.4978647740247766
RMSE of Lasso Regression for Clothing and footwear: 0.2183598730725862
RMSE of Lasso Regression for Communication: 0.18823529411763218
RMSE of Lasso Regression for Education: 0.27408521636726846
RMSE of Lasso Regression for Food and non-alcoholic beverages: 0.23427233386446744
RMSE of Lasso Regression for Headline_CPI: 0.3506674322967598
RMSE of Lasso Regression for Health: 0.29254809357426836
RMSE of Lasso Regression for Household contents and services: 0.11331912097216446
RMSE of Lasso Regression for Housing and utilities: 0.7631320973497964
RMSE of Lasso Regression for Miscellaneous goods and services: 0.018624071199482728
RMSE of Lasso Regression for Recreation and culture: 0.2618507558022003
RMSE of Lasso Regression for Restaurants and hotels : 0.052578421557896604
RMSE of Lasso Regression for Transport: 0.19887695312499432
Average RMSE of Lasso Regression: 0.26649341825571493


In [39]:
X_train = train[features]
X_traint = train[featurest]
y_train = train[target_cols]

X_val = test[features]
X_valt = test[featurest]
y_val = test[target_cols]

l_models = {}

y_predl = []

scaler = MinMaxScaler()

#training
for target_col in target_cols:
    if target_col == "Transport":
        #l_model = Lasso(alpha=0.1, random_state=seed)
        l_model = XGBRegressor(seed=seed)
        X_train_scaled = scaler.fit_transform(X_traint)

        l_model.fit(X_train_scaled, y_train[target_col])

        l_models[target_col] = l_model
        l_model = l_models[target_col]

        X_val_scaled = scaler.transform(X_valt)

        y_pred_coll = l_model.predict(X_val_scaled)

        y_predl.append(y_pred_coll)
    else:
        l_model = Lasso(alpha=0.1, random_state=seed)

        X_train_scaled = scaler.fit_transform(X_train)

        l_model.fit(X_train_scaled, y_train[target_col])

        l_models[target_col] = l_model
        l_model = l_models[target_col]

        X_val_scaled = scaler.transform(X_val)

        y_pred_coll = l_model.predict(X_val_scaled)

        y_predl.append(y_pred_coll)

y_predl = np.array(y_predl).T
y_predl[:,5][0] = ((6.26 * y_predl[:,0][0]) + (3.65 * y_predl[:,1][0]) + (2.42 * y_predl[:,2][0]) 
                   + (2.62 * y_predl[:,3][0]) + (17.14 * y_predl[:,4][0]) + (1.44 * y_predl[:,6][0]) + 
                   (4.37 * y_predl[:,7][0]) 
                   + (24.49 * y_predl[:,8][0]) + (14.81 * y_predl[:,9][0]) + (5.2 * y_predl[:,10][0]) + 
                   (3.25 * y_predl[:,11][0]) + (14.35 * y_predl[:,12][0]))/100

print(f'prediction of Lasso Regression: {y_predl}') 

prediction of Lasso Regression: [[111.10120058 103.91123811  99.67777778 110.42428262 118.53047594
  110.36554632 110.53605939 107.77939479 107.05225307 109.63764282
  104.60344954 110.23367622 113.01968384]]


In [40]:
def prepSub(y_pred:list, target_cols: list, test, prefix:str):
    sub_df = pd.DataFrame(y_pred, columns=target_cols)
    sub_df['Month'] = test['Month']
    
    sub_df.set_index('Month', inplace=True)
    sub_df.columns = [prefix+'_' + col.lower().replace('_', ' ').strip() for col in sub_df.columns]
    sub_df.rename(columns= {f"{prefix}_headline cpi": f"{prefix}_headline CPI"}, inplace=True)
    
    sub_df = pd.melt(sub_df.reset_index(), id_vars= ['Month'], var_name= 'ID', value_name= 'Value')
    
    return sub_df[['ID', 'Value']]

In [41]:
sub = prepSub(y_predl, target_cols, test, 'August') 
sub.to_csv('out/lasso_xgb_with_lag5_historic_cpi_petrol_lending.csv', index=False)